In [1]:
!git clone https://github.com/Hassan-Tamer/UWIE-sonar.git

Cloning into 'UWIE-sonar'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 84 (delta 23), reused 72 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (84/84), 4.68 MiB | 18.28 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp /content/drive/MyDrive/dataset1.zip .
!unzip dataset1.zip

Streaming output truncated to the last 5000 lines.
  inflating: Shipwreck_dataset/camera/c6731s286.jpg  
  inflating: Shipwreck_dataset/camera/c6633s236.jpg  
  inflating: Shipwreck_dataset/camera/c11748s148.jpg  
  inflating: Shipwreck_dataset/camera/c3317s976.jpg  
  inflating: Shipwreck_dataset/camera/c3826s1154.jpg  
  inflating: Shipwreck_dataset/camera/c10013s447.jpg  
  inflating: Shipwreck_dataset/camera/c2937s789.jpg  
  inflating: Shipwreck_dataset/camera/c2385s536.jpg  
  inflating: Shipwreck_dataset/camera/c11609s108.jpg  
  inflating: Shipwreck_dataset/camera/c20623s1087.jpg  
  inflating: Shipwreck_dataset/camera/c527s113.jpg  
  inflating: Shipwreck_dataset/camera/c2626s658.jpg  
  inflating: Shipwreck_dataset/camera/c9795s325.jpg  
  inflating: Shipwreck_dataset/camera/c6756s304.jpg  
  inflating: Shipwreck_dataset/camera/c6764s309.jpg  
  inflating: Shipwreck_dataset/camera/c8513s852.jpg  
  inflating: Shipwreck_dataset/camera/c48s13.jpg  
  inflating: Shipwreck_datase

In [1]:
%cd /content/UWIE-sonar

/content/UWIE-sonar


In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import os
import sys
import copy
import csv
from dataset.dataset import Shipwreck
from net.customNetworks import *
from net.Ushape_Trans import *
from net.utils import *

In [12]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 77.9 MB/s eta 0:00:00


In [16]:
batch_size = 1
lr = 0.001
num_epochs = 10

In [12]:
shipwreck_train = Shipwreck('_','/content/yarab/sonar')
trainloader = torch.utils.data.DataLoader(shipwreck_train, batch_size=batch_size, shuffle=True)
print(len(shipwreck_train))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

1
cuda


In [14]:
!cp /content/Shipwreck_dataset/camera/c0s0.jpg /content/yarab/camera

In [5]:
DeitAE = DeiTAutoencoder().to(device)

Using cache found in /root/.cache/torch/hub/facebookresearch_deit_main
/root/.cache/torch/hub/facebookresearch_deit_main/models.py:63: UserWarning: Overwriting deit_tiny_patch16_224 in registry with models.deit_tiny_patch16_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def deit_tiny_patch16_224(pretrained=False, **kwargs):
/root/.cache/torch/hub/facebookresearch_deit_main/models.py:78: UserWarning: Overwriting deit_small_patch16_224 in registry with models.deit_small_patch16_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def deit_small_patch16_224(pretrained=False, **kwargs):
/root/.cache/torch/hub/facebookresearch_deit_main/models.py:93: UserWarning: Overwriting deit_base_patch16_224 in registry with models.deit_base_patch16_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.

In [20]:
optimizer = torch.optim.Adam(DeitAE.parameters(), lr=0.1,  betas=(0.5, 0.999))
MSE = nn.MSELoss(size_average=False).to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [7]:
def write_row_to_csv(filename, row):
  with open(filename, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(row)

In [ ]:
csv_dir = '/content/drive/MyDrive/trainAE/results/results.csv'
if not os.path.exists(csv_dir):
    write_row_to_csv(csv_dir, ['Epoch', 'Loss'])

visualize_frequency = 15
checkpoint_frequency = 1  # Save a checkpoint every 5 epochs
num_epochs = 55
for epoch in range(13,num_epochs):
    print(f"Epoch [{epoch+1}/{num_epochs}]")

    epoch_loss = 0

    for i, (_,sonar) in enumerate(tqdm(trainloader, desc=f"Epoch {epoch+1}", unit="batch", leave=False)):
        optimizer.zero_grad()

        sonar = sonar.to(device)
        fake = DeitAE(sonar)

        loss = MSE(fake, sonar)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()

        if (i + 1) % visualize_frequency == 0:
            tqdm.write(f"\r[Epoch {epoch+1}/{num_epochs}] [Batch {i+1}/{len(trainloader)}] Loss Pixel: {loss.item():.4f}, ")

        save_dir = f'/content/drive/MyDrive/trainAE/results/{epoch+1}'
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        if i == len(trainloader) - 1:
            save_image(sonar, f"{save_dir}/gt_epoch{epoch+1}.png", normalize=True)
            save_image(fake, f"{save_dir}/fake_epoch{epoch+1}.png", normalize=True)

    # Print epoch summary
    avg_epoch_loss = epoch_loss / len(trainloader)

    print(f"Average Generator Loss: {avg_epoch_loss:.4f}")
    write_row_to_csv(csv_dir, [epoch + 1, avg_epoch_loss])

    if (epoch + 1) % checkpoint_frequency == 0:
        checkpoint_path = f'/content/drive/MyDrive/trainAE/checkpoints/epoch_{epoch+1}.pth'
        torch.save({
            'model_state_dict': DeitAE.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, checkpoint_path)


Epoch [14/55]


Average Generator Loss: 6963.9199
Epoch [15/55]


Average Generator Loss: 6939.7524
Epoch [16/55]


Average Generator Loss: 6929.8018


Epoch [17/55]


Average Generator Loss: 6914.1025
Epoch [18/55]


Average Generator Loss: 6902.0054
Epoch [19/55]


Average Generator Loss: 6890.9014
Epoch [20/55]


Average Generator Loss: 6879.9316


In [27]:
save_image(sonar, f"{save_dir}/gt_epoch{epoch+1}.png", normalize=True)
save_image(fake, f"{save_dir}/fake_epoch{epoch+1}.png", normalize=True)

tensor(1.3270, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0')
